In [173]:
from wilcoxon.spiderman import *
import re
import pandas as pd
from tqdm import tqdm

In [246]:
site = website("https://www.tutorialspoint.com/dax_functions/dax_functions_other.htm")

In [247]:
links = ["https://www.tutorialspoint.com" + x['href'] for x in site.html('a',string=re.compile("function$"))]

In [248]:
def getData(link):

    subsite = website(link)
    software = "Power BI"
    explanation = subsite.html.find("h2",text="Description").find_next("p").text
    subject = list(subsite.html.find("h2",text="Syntax").find_next("pre").stripped_strings)[0]

    para = pd.read_html(str(subsite.html.find("h2",text="Parameters").find_next("table")))[0]
    para['Subject'] = para.iloc[:,1].apply(lambda x: re.split(r"\s(?=[A-Z])",x,1)[0])
    para['Explanation'] = para.iloc[:,1].apply(lambda x: re.split(r"\s(?=[A-Z])",x,1)[1])

    para = para.iloc[:,2:]
    para = para.to_dict("records")

    for x in para:
        x["Software"] = software
        x["Type"] = "parameter"
        x["Link"] = link
        x["Subject"] = re.findall("(.*)\s\(", subject)[0] + f"(<{x['Subject']}>)"

    returnVal = subsite.html.find("h2",text="Return Value").find_next("p").text
    example = list(subsite.html.find("h2",text="Example").find_next("pre").stripped_strings)[0]
    example = example[2:] if example[0] == "=" else example
    remarks = subsite.html.find("h2",text="Remarks").find_next("p").text

    firstRow = [{"Software":software,"Type":"function","Subject":subject,"Explanation":explanation,"Example":example,"Remarks":remarks,"Return":returnVal,"Link":link}]
    df = pd.DataFrame(firstRow,columns = ['Software', 'Subject', 'Type', 'Explanation', 'Return', 'Example', 'Remarks', 'Package', 'Level', 'Link'])
    df = df.append(para)
    
    return df


In [249]:
dfs = []
for link in tqdm(links):
    try:
        dfs.append(getData(link))
    except Exception as e:
        print(link)
        print(e)

100%|██████████| 8/8 [00:04<00:00,  1.78it/s]

https://www.tutorialspoint.com/dax_functions/dax_var_function.htm
list index out of range


In [250]:
pd.concat(dfs).to_clipboard()

In [201]:
para.iloc[:,1]

0    table The table over which all context filters...
1    column One or more columns that are specified ...
Name: Parameter & Description, dtype: object